In [ ]:
%matplotlib inline

import json
import pickle
import pandas
import numpy
import matplotlib.pyplot as plt
import math
from scipy.optimize import curve_fit

from pandas import DataFrame


def log_as_dataframe(log):
    columns = ['timestamp', 'level', 'source', 'function', 'data']
    df = pandas.DataFrame(log, columns=columns)
    return df.set_index('timestamp').sort_index()


df = log_as_dataframe(pickle.load(open('../log.pkl', 'rb')))
df = df[df['function'] == 'log_front_sensors_calibration']
df = DataFrame([json.loads(x) for x in df['data'].values])
df['left_raw'] = df['left_raw_on'] - df['left_raw_off']
df['right_raw'] = df['right_raw_on'] - df['right_raw_off']
df['micrometers'] = df['micrometers'] - df['micrometers'][0]
df['micrometers'] = (180 * 2 - 12 - 36) * 1000 - df['micrometers']
df = df[(df['micrometers'] > 80000) & (df['micrometers'] < 270000)]
df = df.set_index('micrometers')
df = df.sort_index(ascending=False)
df.head()

In [ ]:
distance = df.index.values / 1000000
samples = list(range(len(df.index)))

In [ ]:
x = numpy.array(distance[::])
y_FR = numpy.array(df['right_raw'][::])
y_FL = numpy.array(df['left_raw'][::])
plt.plot(x, y_FL)
plt.show()

In [ ]:
popt_FR, pcov_FR = curve_fit(lambda t,a,b: numpy.exp(a/(b+t)),  x,  y_FR)
popt_FL, pcov_FL = curve_fit(lambda t,a,b: numpy.exp(a/(b+t)),  x,  y_FL)

In [ ]:
f,(ax1,ax2) = plt.subplots(2, sharex=True, sharey= True)
ax1.plot(distance,df['right_raw'])
ax2.plot(distance,df['left_raw'])
def fit_FR(x):
    return numpy.exp(popt_FR[0]/(popt_FR[1]+x))
ax1.plot(x, fit_FR(x))
def fit_FL(x):
    return numpy.exp(popt_FL[0]/(popt_FL[1]+x))
ax2.plot(x, fit_FL(x))
plt.show()
print('FR_a {}, FR_b {}'.format(popt_FR[0],popt_FR[1]))
print('FL_a {}, FL_b {}'.format(popt_FL[0],popt_FL[1]))

In [ ]:
df['front_left_distance'].plot()

In [ ]:
df['left_raw_on'].plot()

In [ ]:
df['left_raw_on'].head(50)

In [ ]:
df['right_raw_on'].plot()